In [6]:
%pip install transformers_stream_generator
%pip install accelerate
%pip install tiktoken
%pip install einops

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Agent
from transformers.generation import GenerationConfig


class QWenAgent(Agent):
    """
    Agent that uses QWen model and tokenizer to generate code.

    Args:
        chat_prompt_template (`str`, *optional*):
            Pass along your own prompt if you want to override the default template for the `chat` method. Can be the
            actual prompt template or a repo ID (on the Hugging Face Hub). The prompt should be in a file named
            `chat_prompt_template.txt` in this repo in this case.
        run_prompt_template (`str`, *optional*):
            Pass along your own prompt if you want to override the default template for the `run` method. Can be the
            actual prompt template or a repo ID (on the Hugging Face Hub). The prompt should be in a file named
            `run_prompt_template.txt` in this repo in this case.
        additional_tools ([`Tool`], list of tools or dictionary with tool values, *optional*):
            Any additional tools to include on top of the default ones. If you pass along a tool with the same name as
            one of the default tools, that default tool will be overridden.

    Example:

    ```py
    agent = QWenAgent()
    agent.run("Draw me a picture of rivers and lakes.")
    ```
    """
    def __init__(self, chat_prompt_template=None, run_prompt_template=None, additional_tools=None):
        checkpoint = "Qwen/Qwen-7B-Chat"
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", trust_remote_code=True).cuda().eval()
        self.model.generation_config = GenerationConfig.from_pretrained(checkpoint, trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参
        self.model.generation_config.do_sample = False  # greedy
        
        super().__init__(
            chat_prompt_template=chat_prompt_template,
            run_prompt_template=run_prompt_template,
            additional_tools=additional_tools,
        )

    def generate_one(self, prompt, stop):
        # "Human:" 和 "Assistant:" 曾为通义千问的特殊保留字，需要替换为 "_HUMAN_:" 和 "_ASSISTANT_:"。这一问题将在未来版本修复。
        prompt = prompt.replace("Human:", "_HUMAN_:").replace("Assistant:", "_ASSISTANT_:")
        stop = [item.replace("Human:", "_HUMAN_:").replace("Assistant:", "_ASSISTANT_:") for item in stop]

        result, _ = self.model.chat(self.tokenizer, prompt, history=None)
        for stop_seq in stop:
            if result.endswith(stop_seq):
                result = result[: -len(stop_seq)]

        result = result.replace("_HUMAN_:", "Human:").replace("_ASSISTANT_:", "Assistant:")
        return result


agent = QWenAgent()
agent.run("Draw me a picture of rivers and lakes.")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/felinejtd/anaconda3/envs/thesis/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/felinejtd/anaconda3/envs/thesis/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/felinejtd/anaconda3/envs/thesis/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

In [3]:
%pip install gradio_client

Note: you may need to restart the kernel to use updated packages.


In [5]:
from gradio_client import Client

client = Client("abidlabs/whisper-large-v2")  # connecting to a Hugging Face Space
client.predict("test.mp4", api_name="/predict")

Loaded as API: https://abidlabs-whisper-large-v2.hf.space ✔


'\n..'

In [2]:
from gradio_client import Client

client = Client("Qwen/Qwen2-VL")
result = client.predict(
		history=[],
		text="Hello!!",
		api_name="/add_text"
)
print(result)

Loaded as API: https://qwen-qwen2-vl.hf.space ✔
[['Hello!!', None]]


In [3]:
from gradio_client import Client, handle_file

result = client.predict(
		history=[],
		file=handle_file('https://github.com/gradio-app/gradio/raw/main/test/test_files/sample_file.pdf'),
		api_name="/add_file"
)
print(result)

[[{'file': '/tmp/gradio/c51f8dda6ee8a458f4a5a8e53e19c98393f66f84c8e4c199a6b859782db3a97a/sample_file.pdf', 'alt_text': None}, None]]


In [4]:
from gradio_client import Client

result = client.predict(
		history=[],
		text="Tell me what you see in that image.",
		api_name="/add_text"
)
print(result)

[['Tell me what you see in that image.', None]]


In [9]:
from gradio_client import Client, file

client = Client("Qwen/Qwen2-VL")

client.predict(
  api_name="/reset_state"
)

client.predict(
  history=[],
  text=None,
  api_name="/add_text"
)

client.predict(
  api_name="/reset_user_input"
)

client.predict(
  _chatbot=[["Hello",None]],
  api_name="/predict"
)


Loaded as API: https://qwen-qwen2-vl.hf.space ✔


AppError: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().